In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import importlib
import plotly.express as px
from datetime import datetime

sys.path.append('../lib')
sys.path.append('/home/jupyter/pensjon-data-analyse/lib')
import utils
import pandas_utils
import pesys_utils

importlib.reload(utils)
importlib.reload(pandas_utils)
importlib.reload(pesys_utils)

utils.set_secrets_as_env()

In [ ]:
con = pesys_utils.open_pen_connection()

df_kravstatus = pandas_utils.pandas_from_sql('/home/jupyter/pensjon-data-analyse/sql/kravstatus.sql', con)
con.close()
df_kravstatus.columns = map(str.lower, df_kravstatus.columns)
df_kravstatus["dato"] = datetime.utcnow()

In [ ]:
df = df_kravstatus.copy()

df.head()

In [ ]:
df_uten_type = df.groupby(["dato", "sakstype", "kravstatus"], as_index=False).antall.sum()

In [ ]:
for sakstype in set(df.sakstype):
    df_plot = df_uten_type[df_uten_type.sakstype == sakstype]
    px.bar(df_plot, "dato", "antall", color="kravstatus", barmode="stack", title=sakstype).show()

In [ ]:
from google.cloud.bigquery import Client, LoadJobConfig, SchemaField, enums

table_id = f'pensjon-saksbehandli-prod-1f83.saksstatistikk.kravstatus'
job_config = LoadJobConfig(
    schema = [
        SchemaField("sakstype", enums.SqlTypeNames.STRING),
        SchemaField("kravtype", enums.SqlTypeNames.STRING),
        SchemaField("kravstatus", enums.SqlTypeNames.STRING),
        SchemaField("antall", enums.SqlTypeNames.INTEGER),
        SchemaField("dato", enums.SqlTypeNames.TIMESTAMP),
    ],
    write_disposition="WRITE_APPEND",
)

client = Client(project="pensjon-saksbehandli-prod-1f83")

job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
job.result()